In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns 

In [ ]:
#L'objectif de cette NoteBook est de tester directement les modéles
#les opérations suivantes sont déja faites précédement dans mon ordinateur

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/test.csv')
sub = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/sample_submission.csv')

In [ ]:
train.drop(['id'],inplace=True,axis=1)

In [ ]:
test.drop(['id'],inplace=True,axis=1)

In [ ]:
#on va essayer de créer des variables plus corrélés a la variable "target"
train['cont3&4'] = train['cont3'] + train['cont4']
train['cont7&8'] = train['cont7'] + train['cont8']
train['cont11&12'] = train['cont11'] + train['cont12']
test['cont3&4'] = test['cont3'] + test['cont4']
test['cont7&8'] = test['cont7'] + test['cont8']
test['cont11&12'] = test['cont11'] + test['cont12']

train['somme1'] = train['cont1'] + train['cont2'] + train['cont3'] + train['cont4'] 
train['somme2'] = train['cont5'] + train['cont6'] + train['cont7'] + train['cont7'] 

test['somme1'] = test['cont1'] + test['cont2'] + test['cont3'] + test['cont4'] 
test['somme2'] = test['cont5'] + test['cont6'] + test['cont7'] + test['cont7'] 



In [ ]:
train['v2'] = 0
for i in range(train.shape[0]):
  if train.cont2.iloc[i]  > 0.5 :
    train.v2.iloc[i] = 1

test['v2'] = 0
for i in range(test.shape[0]):
  if( test.cont2.iloc[i]  > 0.5 ):
    test.v2.iloc[i] = 1

In [ ]:
train.drop(['v1'],axis=1,inplace=True)

In [ ]:
from sklearn.preprocessing import StandardScaler
numerical_cols = ['cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7', 'cont8',
       'cont9', 'cont10', 'cont11', 'cont12', 'cont13', 'cont14',
       'cont3&4', 'cont7&8', 'cont11&12', 'somme1', 'somme2', 'v2']
for c in numerical_cols:
    prep = StandardScaler()
    train[c] = prep.fit_transform(train[[c]])
    test[c] = prep.transform(test[[c]])

In [ ]:
x = train.drop(['target'],axis=1)
y = train['target']
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size=0.15,random_state=42)

In [ ]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout

In [ ]:
model1 = Sequential()
model1.add(Dense(128,'relu'))
model1.add(Dropout(0.5))
model1.add(Dense(64,'relu'))
model1.add(Dropout(0.3))
model1.add(Dense(32,'relu'))
model1.add(Dropout(0.2))
model1.add(Dense(16,'relu'))
model1.add(Dropout(0.1))
model1.add(Dense(1,'linear'))

model1.compile(optimizer='adam',loss='mse',metrics=[keras.metrics.RootMeanSquaredError()])


In [ ]:
model1.fit(
        x_train, y_train,
        validation_data=(x_test, y_test),
        batch_size=10000,
        epochs=1500,
    )

In [ ]:
y_predd = model1.predict(test) 

In [ ]:
#submission
sub['target']= y_predd
sub.to_csv('submission3.csv', index=False)

In [ ]:
import os 
os.chdir(r'../working')
from IPython.display import FileLink
FileLink(r'submission3.csv')